In [1]:
import pandas as pd
#math
from string import punctuation
from itertools import product
import numpy as np

# TIME SERIES
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from statsmodels.tsa.seasonal import seasonal_decompose
#encoder
from sklearn.preprocessing import LabelEncoder
#sys
import time
import calendar
import datetime
import warnings
warnings.filterwarnings("ignore")

# Visualize
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots
from pylab import rcParams
#import calmap
#import squarify

In [2]:
# Global variable
path = "/Users/williamchen/Documents/Kaggle/predict_future_sales/predict_future_sales_data/"

d = {0:57, 1:58, 10:11}

In [3]:
%%time
sales = pd.read_csv(path + "sales_train.csv")
cats = pd.read_csv(path + "item_categories.csv")
items = pd.read_csv(path + "items.csv")
shops = pd.read_csv(path + "shops.csv")
test = pd.read_csv(path+"test.csv",index_col='ID')

CPU times: user 1.21 s, sys: 211 ms, total: 1.42 s
Wall time: 1.43 s


## Feature preprocessing
### Preparing Shop Information
#### Correct the duplicated shop name that found by EDA.
#### Keep the shop_category if there are 5 or more shops of that category, the rest are grouped as "other"

In [4]:
for var in [sales,test,shops]:
    var["shop_id"] = var["shop_id"].apply(lambda x: d[x] if x in d.keys() else x)

In [5]:
# we have seen in our EDA that we have some duplicate shops, let's correct them.
shops.loc[ shops.shop_name == 'Сергиев Посад ТЦ "7Я"',"shop_name" ] = 'СергиевПосад ТЦ "7Я"'
shops["city"] = shops.shop_name.str.split(" ").map( lambda x: x[0] )
shops["category"] = shops.shop_name.str.split(" ").map( lambda x: x[1] )
shops.loc[shops.city == "!Якутск", "city"] = "Якутск"
category = []
for cat in shops.category.unique():
    if len(shops[shops.category == cat]) >= 5:
        category.append(cat)
shops.category = shops.category.apply( lambda x: x if (x in category) else "other" )
from sklearn.preprocessing import LabelEncoder
shops["shop_category"] = LabelEncoder().fit_transform( shops.category )
shops["city_id"] = LabelEncoder().fit_transform( shops.city )

shops = shops[["shop_id", "shop_category", "city_id","city"]]
shops = shops[~shops.shop_id.duplicated()]
shops.sample(5)

,shop_id,shop_category,city_id,city
27,27,4,13,Москва
32,32,4,13,Москва
10,11,0,6,Жуковский
26,26,4,13,Москва
38,38,4,17,Омск


In [6]:
city_population = {\
'Якутск':307911, 
'Адыгея':141970,
'Балашиха':450771, 
'Волжский':326055, 
'Вологда':313012, 
'Воронеж':1047549,
'Выездная':1228680, 
'Жуковский':107560, 
'Интернет-магазин':1228680, 
'Казань':1257391, 
'Калуга':341892,
'Коломна':140129,
'Красноярск':1083865, 
'Курск':452976, 
'Москва':12678079,
'Мытищи':205397, 
'Н.Новгород':1252236,
'Новосибирск':1602915 , 
'Омск':1178391, 
'РостовНаДону':1125299, 
'СПб':5398064, 
'Самара':1156659,
'СергиевПосад':104579, 
'Сургут':373940, 
'Томск':572740, 
'Тюмень':744554, 
'Уфа':1115560, 
'Химки':244668,
'Цифровой':1228680, 
'Чехов':70548, 
'Ярославль':608353
}

city_income = {\
'Якутск':70969, 
'Адыгея':28842,
'Балашиха':54122, 
'Волжский':31666, 
'Вологда':38201, 
'Воронеж':32504,
'Выездная':46158, 
'Жуковский':54122, 
'Интернет-магазин':46158, 
'Казань':36139, 
'Калуга':39776,
'Коломна':54122,
'Красноярск':48831, 
'Курск':31391, 
'Москва':91368,
'Мытищи':54122, 
'Н.Новгород':31210,
'Новосибирск':37014 , 
'Омск':34294, 
'РостовНаДону':32067, 
'СПб':61536, 
'Самара':35218,
'СергиевПосад':54122, 
'Сургут':73780, 
'Томск':43235, 
'Тюмень':72227, 
'Уфа':35257, 
'Химки':54122,
'Цифровой':46158, 
'Чехов':54122, 
'Ярославль':34675
}

shops["city_population"] = shops["city"].map(city_population)
shops["city_income"] = shops["city"].map(city_income)

### Preparing Item/Category Information
#### We only keep type_code if there are 5 or more items of that category, the rest are grouped as "etc".

In [7]:
%%time
cats['split'] = cats['item_category_name'].str.split('-')
cats['type_code'] = cats['split'].map(lambda x: x[0].strip())
cats.loc[(cats.type_code == "Игровые")| (cats.type_code == "Аксессуары"), "type_code"] = "Игры"
category = []
for cat in cats.type_code.unique():
    if len(cats[cats.type_code == cat]) >= 5: 
        category.append(cat)
cats['type_code'] = cats.type_code.apply(lambda x: x if (x in category) else "etc")
cats['type_code'] = LabelEncoder().fit_transform(cats.type_code)
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code','subtype_code']]
item_cats = pd.merge(cats,items, on=["item_category_id"], how='left')
item_cats.drop(['item_name'],axis = 1,inplace = True)

CPU times: user 23.5 ms, sys: 12.8 ms, total: 36.4 ms
Wall time: 34.7 ms


### Preparing sales information
#### Correct the item_price in 2 rows.

In [8]:
%%time
sales.loc[sales.item_price == min(sales.item_price),'item_price'] = 1249.0
sales.loc[sales.item_price == max(sales.item_price),'item_price'] = 1299.0
sales = sales[(sales.item_price < 100000) & (sales.item_price > 0)]
sales = sales[sales.item_cnt_day < 1001]
sales['revenue'] = sales['item_cnt_day']*sales['item_price']

CPU times: user 923 ms, sys: 94.7 ms, total: 1.02 s
Wall time: 996 ms


In [9]:
%%time
# ref https://www.kaggle.com/dlarionov/feature-engineering-xgboost
matrix = []
cols  = ["date_block_num", "shop_id", "item_id"]
for i in range(34):
    tmp = sales[sales.date_block_num == i]
    matrix.append( np.array(list( product( [i], tmp.shop_id.unique(), tmp.item_id.unique() ) ), dtype = np.int16) )

matrix = pd.DataFrame( np.vstack(matrix), columns = cols )
matrix["date_block_num"] = matrix["date_block_num"].astype(np.int8)
matrix["shop_id"] = matrix["shop_id"].astype(np.int8)
matrix["item_id"] = matrix["item_id"].astype(np.int16)
matrix.sort_values( cols, inplace = True )

group = sales.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=cols, how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)
                                .clip(0,20)
                                .astype(np.float16))

# adding the month number to the test set
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)

matrix = pd.concat([matrix, test], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True)

CPU times: user 23.6 s, sys: 2.73 s, total: 26.3 s
Wall time: 26.3 s


In [10]:
matrix.shape

(11128050, 4)

In [11]:
%%time
# merging the dataframes that we preparing
matrix = pd.merge(matrix, shops, on=['shop_id'], how='left')
matrix = pd.merge(matrix, item_cats, on=['item_id'], how='left')
#matrix = pd.merge(matrix, sales_month, on=['date_block_num','shop_id','item_id'], how='left')
matrix.sample(5)

CPU times: user 5.35 s, sys: 1.74 s, total: 7.09 s
Wall time: 7.2 s


,date_block_num,shop_id,item_id,item_cnt_month,shop_category,city_id,city,city_population,city_income,item_category_id,type_code,subtype_code
8315619,23,42,20884,2.0,1,19,СПб,5398064,61536,72,6,59
4229372,11,21,9016,0.0,0,13,Москва,12678079,91368,40,3,4
1306988,3,28,13876,0.0,4,13,Москва,12678079,91368,55,5,2
4782,0,2,12943,0.0,4,0,Адыгея,141970,28842,40,3,4
681085,1,51,14746,1.0,4,24,Тюмень,744554,72227,38,3,1


In [12]:
#matrix['revenue'] = (sales_month['revenue']
#                                 .fillna(0)
#                                 .astype(np.float16))
#matrix["revenue_over_income"] = matrix["revenue"]/matrix["city_income"]

In [13]:
matrix.head()

,date_block_num,shop_id,item_id,item_cnt_month,shop_category,city_id,city,city_population,city_income,item_category_id,type_code,subtype_code
0,0,2,19,0.0,4,0,Адыгея,141970,28842,40,3,4
1,0,2,27,1.0,4,0,Адыгея,141970,28842,19,2,10
2,0,2,28,0.0,4,0,Адыгея,141970,28842,30,0,55
3,0,2,29,0.0,4,0,Адыгея,141970,28842,23,2,16
4,0,2,32,0.0,4,0,Адыгея,141970,28842,40,3,4


In [14]:
# lagging features
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

# lagged feature for item with id one less than current item
def lag_feature_adv(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)+'_adv']
        shifted['date_block_num'] += i
        shifted['item_id'] -= 1
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
        df[col+'_lag_'+str(i)+'_adv'] = df[col+'_lag_'+str(i)+'_adv'].astype('float16')
    return df

In [15]:
%%time
# generating lagged features
matrix = lag_feature(matrix, [1, 2, 3, 6, 12], 'item_cnt_month')
#matrix = lag_feature(matrix, [1], 'revenue_over_income')
#matrix = lag_feature_adv(matrix, [1, 2, 3, 6, 12], 'item_cnt_month')

CPU times: user 37.6 s, sys: 10.4 s, total: 47.9 s
Wall time: 48.4 s


In [16]:
print(matrix.shape)

(11128050, 17)


In [17]:
matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128050 entries, 0 to 11128049
Data columns (total 17 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   date_block_num         int8   
 1   shop_id                int64  
 2   item_id                int64  
 3   item_cnt_month         float16
 4   shop_category          int64  
 5   city_id                int64  
 6   city                   object 
 7   city_population        int64  
 8   city_income            int64  
 9   item_category_id       int64  
 10  type_code              int64  
 11  subtype_code           int64  
 12  item_cnt_month_lag_1   float16
 13  item_cnt_month_lag_2   float16
 14  item_cnt_month_lag_3   float16
 15  item_cnt_month_lag_6   float16
 16  item_cnt_month_lag_12  float16
dtypes: float16(6), int64(9), int8(1), object(1)
memory usage: 1.0+ GB


In [18]:
def downcast(df):
    #reduce size of the dataframe
    float_cols = [c for c in df if df[c].dtype in ["float64"]]
    int_cols = [c for c in df if df[c].dtype in ['int64']]
    df[float_cols] = df[float_cols].astype('float16')
    df[int_cols] = df[int_cols].astype('int16')
    return df

matrix = downcast(matrix)

In [19]:
def mean_encoded_feature(df, group_by, feature, new_feature, lags=[1]):
    # mean encoding the feature
    group = df.groupby(group_by).agg({feature: ['mean']})
    group.columns = [new_feature]
    #group.reset_index(inplace=True)
    
    df = pd.merge(df, group, on=group_by, how='left')
    df[new_feature] = df[new_feature].astype(np.float16)
    
    # Lagging the mean encoded feature
    df = lag_feature(df, lags, new_feature)
    
    # Removing for the current month
    df.drop([new_feature], axis=1, inplace=True)
    return df

In [20]:
%%time
# mean encoding various features
matrix = mean_encoded_feature(matrix, ['date_block_num'], 'item_cnt_month', 'date_target_enc', [1])
matrix = mean_encoded_feature(matrix, ['date_block_num', 'item_id'], 'item_cnt_month', 'date_item_target_enc', [1])
matrix = mean_encoded_feature(matrix, ['date_block_num', 'shop_id'], 'item_cnt_month', 'date_shop_target_enc', [1])
matrix = mean_encoded_feature(matrix, ['date_block_num', 'item_id', 'shop_id'], 'item_cnt_month', 'date_item_shop_target_enc', [1])
matrix = mean_encoded_feature(matrix, ['date_block_num', 'item_category_id'], 'item_cnt_month', 'date_cat_target_enc', [1])
matrix = mean_encoded_feature(matrix, ['date_block_num', 'shop_id', 'item_category_id'], 'item_cnt_month', 'date_shop_cat_target_enc', [1])
matrix = mean_encoded_feature(matrix, ['date_block_num', 'shop_id', 'type_code'], 'item_cnt_month', 'date_shop_type_target_enc', [1])
matrix = mean_encoded_feature(matrix, ['date_block_num', 'city_id'], 'item_cnt_month', 'date_city_target_enc', [1,2,3])
matrix = mean_encoded_feature(matrix, ['date_block_num', 'item_id', 'city_id'], 'item_cnt_month', 'date_item_city_target_enc', [1])
matrix = mean_encoded_feature(matrix, ['date_block_num', 'type_code'], 'item_cnt_month', 'date_type_target_enc', [1])
matrix = mean_encoded_feature(matrix, ['date_block_num', 'subtype_code'], 'item_cnt_month', 'date_subtype_target_enc', [1])

CPU times: user 3min 5s, sys: 53.9 s, total: 3min 59s
Wall time: 3min 59s


In [21]:
# Total monthly revenue for each shop 
#%%time
group = sales.groupby(['date_block_num','shop_id']).agg({'revenue': ['sum']})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

# total average revenue for each shop
group = group.groupby(['shop_id']).agg({'date_shop_revenue': ['mean']})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['shop_id'], how='left')
matrix['shop_avg_revenue'] = matrix['shop_avg_revenue'].astype(np.float32)

# Monthly revenue difference from the average divided by the average revenue for scaling
matrix['delta_revenue'] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix['delta_revenue'] = matrix['delta_revenue'].astype(np.float16)

# Lagging the feature
matrix = lag_feature(matrix, [1], 'delta_revenue')

matrix.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'], axis=1, inplace=True)

In [22]:
#%%time
# month number from 0 to 11
matrix['month'] = matrix['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])

# days in each month
matrix['days'] = matrix['month'].map(days).astype(np.int8)

# number of weekends each month
def weekends(date_block_num):
    month = date_block_num % 12 + 1
    year = 2013 + date_block_num // 12
    return len([1 for i in calendar.monthcalendar(year, month) if i[6] != 0])
    
matrix['weekends'] = matrix['date_block_num'].apply(lambda x: weekends(x)).astype(np.int8)

In [23]:
#%%time
first_item_block = matrix.groupby(['item_id'])['date_block_num'].min().reset_index()
first_item_block['item_first_interaction'] = 1

# item first bought
first_shop_item_buy_block = matrix[matrix['date_block_num'] > 0].groupby(['shop_id', 'item_id'])['date_block_num'].min().reset_index()
first_shop_item_buy_block['first_date_block_num'] = first_shop_item_buy_block['date_block_num']

matrix = pd.merge(matrix, first_item_block[['item_id', 'date_block_num', 'item_first_interaction']], on=['item_id', 'date_block_num'], how='left')
matrix = pd.merge(matrix, first_shop_item_buy_block[['item_id', 'shop_id', 'first_date_block_num']], on=['item_id', 'shop_id'], how='left')

matrix['first_date_block_num'].fillna(100, inplace=True)
matrix['shop_item_sold_before'] = (matrix['first_date_block_num'] < matrix['date_block_num']).astype('int8')
matrix.drop(['first_date_block_num'], axis=1, inplace=True)

matrix['item_first_interaction'].fillna(0, inplace=True)
matrix['shop_item_sold_before'].fillna(0, inplace=True)

# is it the first time the item appears
matrix['item_first_interaction'] = matrix['item_first_interaction'].astype('int8')  
# has item been sold before
matrix['shop_item_sold_before'] = matrix['shop_item_sold_before'].astype('int8') 


# average category sales for the new item
item_id_target_mean = matrix[matrix['item_first_interaction'] == 1].groupby(['date_block_num','subtype_code'])['item_cnt_month'].mean().reset_index().rename(columns={
    "item_cnt_month": "new_item_cat_avg"}, errors="raise")

matrix = pd.merge(matrix, item_id_target_mean, on=['date_block_num','subtype_code'], how='left')

matrix['new_item_cat_avg'] = (matrix['new_item_cat_avg']
                                .fillna(0)
                                .astype(np.float16))

matrix = lag_feature(matrix, [1, 2, 3], 'new_item_cat_avg')
matrix.drop(['new_item_cat_avg'], axis=1, inplace=True)

# average category sales for the new item in each store
item_id_target_mean = matrix[matrix['item_first_interaction'] == 1].groupby(['date_block_num','subtype_code', 'shop_id'])['item_cnt_month'].mean().reset_index().rename(columns={
    "item_cnt_month": "new_item_shop_cat_avg"}, errors="raise")

matrix = pd.merge(matrix, item_id_target_mean, on=['date_block_num','subtype_code', 'shop_id'], how='left')

matrix['new_item_shop_cat_avg'] = (matrix['new_item_shop_cat_avg']
                                .fillna(0)
                                .astype(np.float16))

matrix = lag_feature(matrix, [1, 2, 3], 'new_item_shop_cat_avg')
matrix.drop(['new_item_shop_cat_avg'], axis=1, inplace=True)

In [24]:
#%%time
# months since first sale in the shop
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
# months since first sale over all shops
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

In [25]:
holidays_next_month = {
    12:8,
    1:1,
    2:1,
    3:0,
    4:2,
    5:1,
    6:0,
    7:0,
    8:0,
    9:0,
    10:1,
    11:0
}
holidays_this_month = {
    1:8,
    2:1,
    3:1,
    4:0,
    5:2,
    6:1,
    7:0,
    8:0,
    9:0,
    10:0,
    11:1,
    12:0
}
matrix['month'] += 1
matrix["holidays_next_month"] = matrix["month"].map(holidays_next_month).astype(np.int8)
matrix["holidays_this_month"] = matrix["month"].map(holidays_this_month).astype(np.int8)

In [26]:
# fill the nan's caused by lagging the features

def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            df[col].fillna(0, inplace=True)         
    return df

matrix = fill_na(matrix)

In [28]:
matrix.drop(['city','days'],axis = 1 ,inplace = True)

In [29]:
# saving all the data
matrix.to_pickle('all_data.pkl')